In [0]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import torch
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

In [0]:
data, target = load_iris(True)
scl = StandardScaler()
data = scl.fit_transform(data)
xtr, xts, ytr, yts = tts(data, target, test_size = 0.5)
ytr, yts = torch.tensor(ytr).long(), torch.tensor(yts).long()
xtr, xts = torch.tensor(xtr).float(), torch.tensor(xts).float()

In [0]:
class Net(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.net = torch.nn.Sequential(
        torch.nn.Linear(4,3),
        torch.nn.SELU(),
        torch.nn.Linear(3,3),
        torch.nn.Softmax(dim = 1)
    )

  def forward(self, x):
    return self.net(x)

In [0]:
class SimAnnealing(torch.optim.Optimizer):
  def __init__(self, params, T = 2.0, p_variance = 0.05, annealing_rate = 0.99, anneal_every=5):
    if T <= 0.0:
      raise RuntimeError('Initial T must be positive!')
    if not ( 0.0 < annealing_rate < 1.0):
      raise RuntimeError('Annealing rate must be between 0 and 1 exclusively!')
    defaults = dict(T = T, anr = annealing_rate)
    self.T = T
    self.anr = annealing_rate
    self.pvar = p_variance
    self.ane = anneal_every
    self.nstep = 0
    super(SimAnnealing, self).__init__(params, defaults)

  def get_T(self):
    return self.T

  def step(self, closure = None):

    def p_star(num, T):
      return num.div_(-T).exp_()

    def fmt(unit):
      return round(unit.item(),5) 
    if not closure:
      raise RuntimeError('SA requires closure for computing acceptance ratio!')
    loss = p_star(closure(), self.T)
    self.nstep += 1

    for group in self.param_groups:
      back_weights = [-1]*len(group['params'])
      for i,p in enumerate(group['params']):
        back_weights[i] = p.data
        p.data = torch.distributions.Normal(p.data, self.pvar).sample()
      new_loss = p_star(closure(), self.T)
      u = torch.distributions.Uniform(0,1).sample()
      if u.item() > new_loss.item()/loss.item():
        for i,p in enumerate(group['params']):
          p.data = back_weights[i]
    if not (self.nstep % self.ane):
      self.T *= self.anr
    return loss

In [58]:
def init_weights(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

SEED = 12345678
torch.manual_seed(SEED)
nn = Net()
nn.apply(init_weights)
opt = SimAnnealing(nn.parameters())
lossfn = torch.nn.CrossEntropyLoss()
i = -1
max_acc = 0.0
max_wieghts_dict = None

while opt.get_T() > 0.003:
  i+=1
  def closure():
        opt.zero_grad()
        pred = nn(xtr)
        loss = lossfn(pred, ytr)
        loss.backward()
        return loss
  try:
    loss = opt.step(closure)  
  except ZeroDivisionError:
    print('Rounded loss zeroed. Stopping!')
    print(f'Best accuracy = {round(max_acc.item(),4)}')
    break
 
  y_pred = nn(xts)
  correct = (torch.argmax(y_pred,dim=1) == yts).float().sum()
  acc = correct/len(yts)
  if acc > max_acc:
    max_acc = acc
    max_wieghts_dict = nn.state_dict()
  if not i % 50 or acc > 0.9:
    print(f"Epoch#{i}: Train energy level = {loss.item()}, test accuracy = {round(acc.item(),4)}")
  if acc > 0.9:
    break

Epoch#0: Train energy level = 0.584721028804779, test accuracy = 0.4667
Epoch#50: Train energy level = 0.6008089184761047, test accuracy = 0.7333
Epoch#100: Train energy level = 0.5920114517211914, test accuracy = 0.76
Epoch#150: Train energy level = 0.5689440965652466, test accuracy = 0.72
Epoch#200: Train energy level = 0.4923363924026489, test accuracy = 0.68
Epoch#250: Train energy level = 0.4139770269393921, test accuracy = 0.44
Epoch#300: Train energy level = 0.4578160345554352, test accuracy = 0.7467
Epoch#350: Train energy level = 0.4415571093559265, test accuracy = 0.7067
Epoch#400: Train energy level = 0.3933446705341339, test accuracy = 0.7333
Epoch#450: Train energy level = 0.35685524344444275, test accuracy = 0.72
Epoch#500: Train energy level = 0.27824288606643677, test accuracy = 0.52
Epoch#550: Train energy level = 0.2541567087173462, test accuracy = 0.64
Epoch#600: Train energy level = 0.1995590478181839, test accuracy = 0.6133
Epoch#650: Train energy level = 0.2113400

In [59]:
torch.manual_seed(SEED)
nn = Net()
nn.apply(init_weights)
opt = torch.optim.Adadelta(nn.parameters())
lossfn = torch.nn.CrossEntropyLoss()
i = 0
max_acc = 0.0
max_wieghts_dict = None
max_iter = 2000

while i <= max_iter:
  i+=1
  def closure():
        opt.zero_grad()
        pred = nn(xtr)
        loss = lossfn(pred, ytr)
        loss.backward()
        return loss
  loss = opt.step(closure)  
  y_pred = nn(xts)
  correct = (torch.argmax(y_pred,dim=1) == yts).float().sum()
  acc = correct/len(yts)
  if acc > max_acc:
    max_acc = acc
    max_wieghts_dict = nn.state_dict()
  if not i % 50 or acc > 0.9:
    print(f"Epoch#{i}: Train loss = {loss.item()}, test accuracy = {round(acc.item(),4)}")
  if acc > 0.9:
    break

Epoch#50: Train loss = 0.8260712623596191, test accuracy = 0.7733
Epoch#100: Train loss = 0.7506687641143799, test accuracy = 0.84
Epoch#136: Train loss = 0.710701584815979, test accuracy = 0.9067
